In [ ]:
# Solution adapted from https://gist.github.com/latkin/45d8e009f471b4b5d609

In [14]:
class Spell:
    def __init__(self, name, cost, damage, heal, armor, mana, duration):
        self.name = name
        self.cost = cost
        self.damage = damage
        self.heal = heal
        self.armor = armor
        self.mana = mana
        self.duration = duration
    def clone(self):
        return type(self)(name=self.name, cost=self.cost, damage=self.damage, heal=self.heal,
                          armor=self.armor, mana=self.mana, duration=self.duration)
    def __eq__(self, other):
        return other.name == self.name
    def __repr__(self):
        return self.name+" "+str(self.duration)

MagicMissile = Spell("MagicMissile", cost=53,  damage=4, heal=0, armor=0, mana=0,   duration=1)
Drain        = Spell("Drain",        cost=73,  damage=2, heal=2, armor=0, mana=0,   duration=1)
Shield       = Spell("Shield",       cost=113, damage=0, heal=0, armor=7, mana=0,   duration=6)
Poison       = Spell("Poison",       cost=173, damage=3, heal=0, armor=0, mana=0,   duration=6)
Recharge     = Spell("Recharge",     cost=229, damage=0, heal=0, armor=0, mana=101, duration=5)

all_spells = [MagicMissile, Drain, Shield, Poison, Recharge]

In [43]:
def play(part, my_turn, best, spent, player, boss):
    hp, mana, spells = player
    boss_hp, boss_damage = boss
    if spent >= best:
        return best
    if part == 2 and my_turn and hp == 1:
        return best
    mana = sum(map(lambda s: s.mana, spells)) + mana
    damage = sum(map(lambda s: s.damage, spells))
    armor = sum(map(lambda s: s.armor, spells))
    boss_hp = boss_hp - damage
    if boss_hp <= 0:
        return spent
    new_spells = []
    for spell in map(lambda s: s.clone(), spells):
        spell.duration -= 1
        if spell.duration > 0:
            new_spells.append(spell)
    spells = new_spells
    if my_turn:
        if part == 2:
            hp -= 1
        new_best = best
        buyable_spells = filter(
            lambda s: s.cost <= mana and s not in spells, [spell.clone() for spell in all_spells])
        for spell in buyable_spells:
            if spell.duration == 1:
                extra_damage_, heal_, spells_ = spell.damage, spell.heal, spells
            else:
                extra_damage_, heal_, spells_ = 0, 0, spells+[spell]
            spent_ = spent+spell.cost
            mana_ = mana - spell.cost
            hp_ = hp + heal_
            boss_hp_ = boss_hp - extra_damage_
            if boss_hp_ <= 0:
                new_best = min(new_best, spent_)
            else:
                new_best = min(new_best, play(part, False, new_best, spent_, (hp_, mana_, spells_), (boss_hp_, boss_damage)))
        return new_best
    else:
        damage = max(boss_damage-armor, 1)
        hp = hp - damage
        if hp <= 0:
            return best
        else:
            return play(part, True, best, spent, (hp, mana, spells), (boss_hp, boss_damage))

print(play(1, True, 9999, 0, (50, 500, []), (58, 9)))
print(play(2, True, 9999, 0, (50, 500, []), (58, 9)))

1269
1309
